# 1. Thực hiện import thư mục lên Google Drive và Import vào Google Colab

Sử dụng 2 thư viện os và google.colab.drive để liên kết tập tin trong MyDrive/HW2_ML vừa upload lên Drive. Force_remount để buộc Colab mount phân vùng lựa chọn trên Google Drive khi hết session hoặc kết nối gặp vấn đề và không truy cập vào được Google Colab.


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
path='/content/drive/MyDrive/HW2_ML/HW2_ML'
for filename in os.listdir(path):
  print(os.path.join(path,filename))
!cd /content/drive/MyDrive/HW2_ML/HW2_ML/


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/HW2_ML/HW2_ML/converter.py
/content/drive/MyDrive/HW2_ML/HW2_ML/reader.py
/content/drive/MyDrive/HW2_ML/HW2_ML/__init__.py
/content/drive/MyDrive/HW2_ML/HW2_ML/tools.py
/content/drive/MyDrive/HW2_ML/HW2_ML/__main__.py
/content/drive/MyDrive/HW2_ML/HW2_ML/unit_test.py
/content/drive/MyDrive/HW2_ML/HW2_ML/softmax_np.py
/content/drive/MyDrive/HW2_ML/HW2_ML/py2nb-master
/content/drive/MyDrive/HW2_ML/HW2_ML/data
/content/drive/MyDrive/HW2_ML/HW2_ML/__pycache__
/content/drive/MyDrive/HW2_ML/HW2_ML/doc
/content/drive/MyDrive/HW2_ML/HW2_ML/util.py
/content/drive/MyDrive/HW2_ML/HW2_ML/.png
/content/drive/MyDrive/HW2_ML/HW2_ML/plot_tf_500.png
/content/drive/MyDrive/HW2_ML/HW2_ML/plot_tf_1000.png
/content/drive/MyDrive/HW2_ML/HW2_ML/plot_tf_1500.png
/content/drive/MyDrive/HW2_ML/HW2_ML/.ipynb_checkpoints
/content/drive/MyDrive/HW2_ML/HW2_ML/plot

#2. Bài 1: Phân loại hai lớp dùng logistic regression
#2.1 Dùng Numpy

[TODO 1.1,1.2] CHUẨN HOÁ DỮ LIỆU ĐẦU VÀO

- Do ảnh đầu vào là ảnh xám (gray images), nên tham số cần tính là mean và std của tập train_x sẽ chỉ cần tính thông qua 2 chiều đầu tiên của tensor (axis=0 và axis=1).
- Đối với phương pháp xem từng pixel của ảnh là một đặc trưng thì sẽ sử dụng axis, còn phương pháp xem toàn bộ các pixel trong 1 ảnh là một đặc trưng thì tính mean và std trên cả tập dữ liệu. 
```
    num_train = train_x.shape[0]
    num_test = test_x.shape[0]
    height = train_x.shape[1]
    width = train_x.shape[2]

    train_mean = np.zeros((height, width))
    train_std = np.zeros((height, width))

    for i in range(height):
        for j in range(width):
            pixel_data = train_x[:, i, j]
            train_mean[i, j] = sum(pixel_data) / num_train
            train_std[i, j] = math.sqrt(sum((pixel_data - train_mean[i, j]) ** 2) / num_train)

            train_x_norm = np.zeros(train_x.shape)

    for i in range(num_train):
        for j in range(height):
            for k in range(width):
                train_x_norm[i, j, k] = (train_x[i, j, k] - train_mean[j, k]) / train_std[j, k]

            test_x_norm = np.zeros(test_x.shape)
    for i in range(num_test):
        for j in range(height):
            for k in range(width):
                test_x_norm[i, j, k] = (test_x[i, j, k] - train_mean[j, k]) / train_std[j, k]
```

```
    mean=np.mean(train_x)
    std=np.std(train_x)
    for i in range(train_x.shape[0]):
        for j in range(train_x.shape[1]):
            for k in range(train_x.shape[2]):
                train_x[i][j][k] = (train_x[i][j][k] - mean) / std

    for i in range(test_x.shape[0]):
        for j in range(test_x.shape[1]):
            for k in range(test_x.shape[2]):
                test_x[i][j][k] = (test_x[i][j][k] - mean) / std
```
[TODO 1.3] RESHAPE TENSOR VỀ 2D


- Dữ liệu đầu vào là 1 Tensor 3D (2400,64,64), cần resize về 1 Tensor 2D (2400,4096) bằng np.reshape. Giá trị -1 để numpy sẽ tự quyết định size cho chiều còn lại khi đã định kích thước chiều đầu tiên là 2400.
```
tensorR =np.reshape(tensor,(tensor.shape[0],tensor.shape[1]*tensor.shape[2]))
```

[TODO 1.4] THÊM ĐẶC TRƯNG 1 VÀO DỮ LIỆU


- Tạo 1 cột với np.ones
- Chèn cột vào tensor vừa reshape dùng np.hstack.

```
 return np.hstack((x, np.ones((x.shape[0], 1))))
```

[TODO 1.5] TÍNH CÁC GIÁ TRỊ PHÂN LOẠI


- Tính giá trị z
- Tính giá trị phân loại y_hat --> Giá trị cần để trả lại
```
        z=np.dot(x,self.w)
        result=1/(1+np.exp(-z))
```
[TODO 1.6] TÍNH ĐỘ LỖI


- Với số mẫu là m=2400, công thức tính loss=-1/m (y*log(y_hat) + (1-y)*log(1-y_hat))

```
        m=len(y)
        loss = -1/m*np.sum(y*np.log(y_hat)+(1-y)*np.log(1-y_hat))
```
[TODO 1.7] TÍNH ĐẠO HÀM GRADIENT


- Cần chuyển vị x -- x.T

```
        m=len(y)
        w_grad = 1/m*np.dot(x.T,(y_hat-y))
```
[TODO 1.8] CẬP NHẬT TRỌNG SỐ WEIGHT THEO SGD


- Cập nhật theo phương pháp SGD: w=w_old-learning_rate*grad
```
      self.w = self.w-learning_rate*grad
```
[TODO 1.9] CẬP NHẬT TRỌNG SỐ WEIGHT THEO MOMENTUM


- Theo đó, với phương pháp Momentum, dùng giá trị gradient tính toán được để tính toán giá trị Momentum và cập nhật weight. Phương pháp này tăng tốc quá trình học tập  và tránh bị rơi vào các điểm cực tiểu nội bộ của kỹ thuật Gradient Descent.
- Với momentum tính như sau:  momentum=momentum_rate*momentum-learning_rate*grad
```
        momentum=momentum_rate*momentum+learning_rate*grad
        self.w -= momentum
```

[TODO 1.10] KIỂM THỬ



True positive (TP) là số lượng các mẫu được phân loại đúng vào lớp positive (lớp 1). False positive (FP) là số lượng các mẫu được phân loại sai vào lớp positive. False negative (FN) là số lượng các mẫu thuộc lớp positive mà bị phân loại sai vào lớp negative.

- Tính toán các giá trị 

TP: y_hat>=0.5&&test_y==1

FP: y_hat>=0.5&&test_y==0

TN: y_hat<0.5&&test_y==1

FN: y_hat<0.5&&test_y==0

Đếm các giá trị trên thành tổng của từng giá trị từ đó ra được các tham số để tính toán Precision, Recall và F1-score

Tối ưu các thông số epoch, learning_rate, momentum_rate, epochs_to_draw thông qua quá trình kiểm thử và chọn ra thông số phù hợp khi kết quả dần trở nên bão hoà


In [8]:
!python /content/drive/MyDrive/HW2_ML/HW2_ML/logistic_np.py
%matplotlib inline

Streaming output truncated to the last 5000 lines.
Epoch 2504: loss is 0.01092
Time: 0.02499331200000654
Epoch 2505: loss is 0.01092
Time: 0.02553324299999815
Epoch 2506: loss is 0.01091
Time: 0.0315786820000028
Epoch 2507: loss is 0.01091
Time: 0.026992027999995116
Epoch 2508: loss is 0.01090
Time: 0.024092822999989494
Epoch 2509: loss is 0.01090
Time: 0.02543276500000502
Epoch 2510: loss is 0.01090
Time: 0.024552107000005208
Epoch 2511: loss is 0.01089
Time: 0.024712684999997236
Epoch 2512: loss is 0.01089
Time: 0.027524012000000653
Epoch 2513: loss is 0.01088
Time: 0.024615496000009784
Epoch 2514: loss is 0.01088
Time: 0.02426954799999237
Epoch 2515: loss is 0.01087
Time: 0.02511200200000019
Epoch 2516: loss is 0.01087
Time: 0.023760323000004746
Epoch 2517: loss is 0.01087
Time: 0.023942577999989112
Epoch 2518: loss is 0.01086
Time: 0.023973107000003324
Epoch 2519: loss is 0.01086
Time: 0.024018983999994248
Epoch 2520: loss is 0.01085
Time: 0.02418082900000229
Epoch 2521: loss is 0.

In [ ]:
!python /content/drive/MyDrive/HW2_ML/HW2_ML/unit_test.py

Please enter 0 for logistic or 1 for softmax: 0
Running logistic np unit test...
Reading vehicle data...
EOF Reached
Done reading
Testing normalize_per_pixel: Passed.
----------------------------------------
Reading vehicle data...
EOF Reached
Done reading
Testing normalize_all_pixel: Passed.
----------------------------------------
Testing reshape2D: Passed.
----------------------------------------
Testing add_one: Passed.
----------------------------------------
Testing feed_forward 1: Passed.
----------------------------------------
Testing compute_loss 1: Passed.
----------------------------------------
Testing get_grad 1: Passed.
----------------------------------------
Testing update_weight 1: Passed.
----------------------------------------
Testing update_weight_momentum 1: Passed.
----------------------------------------
Testing feed_forward 2: Passed.
----------------------------------------
Testing compute_loss 2: Passed.
----------------------------------------
Testing get_g

#2.2 Dùng TensorFlow


[TODO 1.11] tạo TF placeholders cho train_x và train_y.
```
    x=tf2.placeholder(tf.float32,[None,train_x.shape[1]],name='x')
    y=tf2.placeholder(tf.float32,[None,1],name='y')
```

[TODO 1.12] tạo TF variable để chứa w
```
    W=tf.Variable(tf.zeros((train_x.shape[1],1)),dtype=tf.float32,name='W')
```

[TODO 1.13]  tạo toán tử feed forward.
```
    pred=tf.sigmoid(tf.matmul(x,W))
```

[TODO 1.14]  tính cost.
```
cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=pred))
```

[TODO 1.15] tạo đối tượng SGD optimizer
```
optimizer=tf2.train.GradientDescentOptimizer(learning_rate).minimize(cost)
```

[TODO 1.16] tính cost và cập nhật w

```
 _, loss = sess.run([optimizer, cost], feed_dict={x:train_x, y:train_y})
            all_loss.append(loss)
```


In [10]:
!python /content/drive/MyDrive/HW2_ML/HW2_ML/logistic_tf.py

Streaming output truncated to the last 5000 lines.
Epoch 7504: loss is 0.52118
Time: 0.14311432838439941
Epoch 7505: loss is 0.52118
Time: 0.1012105941772461
Epoch 7506: loss is 0.52118
Time: 0.06078457832336426
Epoch 7507: loss is 0.52118
Time: 0.059365034103393555
Epoch 7508: loss is 0.52118
Time: 0.05777549743652344
Epoch 7509: loss is 0.52118
Time: 0.05607342720031738
Epoch 7510: loss is 0.52118
Time: 0.10625672340393066
Epoch 7511: loss is 0.52118
Time: 0.10761022567749023
Epoch 7512: loss is 0.52118
Time: 0.08622217178344727
Epoch 7513: loss is 0.52118
Time: 0.05734419822692871
Epoch 7514: loss is 0.52118
Time: 0.08335208892822266
Epoch 7515: loss is 0.52117
Time: 0.10214376449584961
Epoch 7516: loss is 0.52117
Time: 0.1684703826904297
Epoch 7517: loss is 0.52117
Time: 0.09688568115234375
Epoch 7518: loss is 0.52117
Time: 0.16522860527038574
Epoch 7519: loss is 0.52117
Time: 0.09689044952392578
Epoch 7520: loss is 0.52117
Time: 0.08044314384460449
Epoch 7521: loss is 0.52117
Time

#3. BÀI 2 PHÂN LOẠI NHIỀU LỚP 
#3.1 DÙNG NUMPY

[TODO 2.1]
Apply data scaling on train_x, val_x, and test_x using computed values

```
    train_mean = train_x.mean(axis=(0,1),keepdims=True)
    train_std = train_x.std(axis=(0,1),keepdims=True)

   
    train_x = (train_x - train_mean) / train_std
    val_x = (val_x - train_mean) / train_std
    test_x = (test_x - train_mean) / train_std
```

[TODO 2.2]
Create the one-hot label matrix here based on labels
```
        one_hot_labels=np.eye(num_k)[labels]
```

[TODO 2.3] Compute a feed forward pass
```
        z = np.dot(x, self.w) + self.b
```
[TODO 2.4] Compute categorical loss
```
        num_samples=y.shape[0]
        loss = -(1/num_samples)*np.sum(y * np.log(y_hat))
```

[TODO 2.5] Compute gradient of the loss function with respect to w
```
        m=len(y)
        w_grad=1/m*np.dot(x.T,(y_hat-y))
```

In [33]:
!python /content/drive/MyDrive/HW2_ML/HW2_ML/softmax_np.py

Streaming output truncated to the last 5000 lines.
Epoch:464,time=0.014661,train_loss=1.561052,val_loss=3.311037
Epoch:465,time=0.015355,train_loss=1.558680,val_loss=3.309698
Epoch:466,time=0.015290,train_loss=1.556314,val_loss=3.308363
Epoch:467,time=0.015316,train_loss=1.553956,val_loss=3.307030
Epoch:468,time=0.015660,train_loss=1.551604,val_loss=3.305699
Epoch:469,time=0.015786,train_loss=1.549260,val_loss=3.304371
Epoch:470,time=0.021414,train_loss=1.546923,val_loss=3.303045
Epoch:471,time=0.015585,train_loss=1.544592,val_loss=3.301721
Epoch:472,time=0.015388,train_loss=1.542269,val_loss=3.300400
Epoch:473,time=0.014955,train_loss=1.539952,val_loss=3.299080
Epoch:474,time=0.014959,train_loss=1.537643,val_loss=3.297763
Epoch:475,time=0.015300,train_loss=1.535340,val_loss=3.296447
Epoch:476,time=0.016537,train_loss=1.533045,val_loss=3.295134
Epoch:477,time=0.025952,train_loss=1.530756,val_loss=3.293822
Epoch:478,time=0.015440,train_loss=1.528475,val_loss=3.292512
Epoch:479,time=0.01

#3.2 DÙNG TENSORFLOW

[TODO 2.6] Create TF placeholders to feed train_x and train_y when training

```
    x = tf2.placeholder(tf.float32, [None, train_x.shape[1]], name='x')
    y = tf2.placeholder(tf.float32, [None, train_y.shape[1]], name='y')
```
[TODO 2.7] Create weights (W) using TF variables 
```
    W = tf.Variable(tf.zeros([train_x.shape[1], train_y.shape[1]]), dtype=tf.float32, name='W')
```
[TODO 2.8] Create a feed-forward operator 
```
pred = tf.nn.softmax(tf.matmul(x, W))

```
[TODO 2.9] Write the cost function
```
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
```
[TODO 2.10] Create an SGD optimizer

```
optimizer = tf2.train.GradientDescentOptimizer(learning_rate).minimize(cost)
```
[TODO 2.11] Compute losses and update weights here
```
train_loss = sess.run(cost, feed_dict={x:train_x, y:train_y})
            val_loss = sess.run(cost, feed_dict={x:val_x, y:val_y})
            _, train_loss = sess.run([optimizer, cost], feed_dict={x: train_x, y: train_y})
            val_loss = sess.run(cost, feed_dict={x: val_x, y: val_y})
            # Update weights
            sess.run(optimizer, feed_dict={x: train_x, y: train_y})

```
[TODO 2.12] Define your own stopping condition here
```
if val_loss >= prev_val_loss:
                num_val_increase+=1
            else:
                num_val_increase=0
            prev_val_loss=val_loss
            if num_val_increase>=patience:
                print('Stopping training. Validation loss has not improved for {} epochs.'.format(num_val_increase))
                break
```

In [25]:
!python /content/drive/MyDrive/HW2_ML/HW2_ML/softmax_tf.py

2023-03-01 15:00:10.488296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 15:00:11.835428: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-01 15:00:11.835564: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-01 15:00:11.835587: W tensorfl